In [2]:
import pandas as pd

In [3]:
import sqlite3

In [4]:
property_df = pd.read_csv("HomeHarvest_Chicago_10years_eda.csv")

In [5]:
crime_df = pd.read_csv("/Users/rashidbaset/Documents/my-projects/crime-home-price-prediction-capstone/chicago-crime-property-analysis/crime_2014_2023_eda.csv")

In [6]:
property_df.columns

Index(['property_url', 'style', 'beds', 'full_baths', 'half_baths', 'sqft',
       'year_built', 'days_on_mls', 'list_price', 'list_date', 'sold_price',
       'last_sold_date', 'lot_sqft', 'price_per_sqft', 'latitude', 'longitude',
       'community', 'neighborhood'],
      dtype='object')

In [7]:
crime_df.columns

Index(['date', 'year', 'latitude', 'longitude', 'description',
       'location_description', 'updated_on', 'domestic', 'primary_type',
       'crime_category', 'arrest', 'community', 'neighborhood', 'TOT_POP'],
      dtype='object')

In [8]:
property_df['last_sold_date'] = pd.to_datetime(property_df['last_sold_date'])

# Extract the year from 'last_sold_date' and create a 'year' column
property_df['year'] = property_df['last_sold_date'].dt.year

In [9]:
conn = sqlite3.connect('chicago-crime-property-6.db')
cursor = conn.cursor()



In [10]:
# # Convert other columns to the desired data types as before
# #crime_df['district'] = crime_df['district'].fillna(-1).astype(int)
# #crime_df['y_coordinate'] = crime_df['y_coordinate'].astype(float)
# #crime_df['x_coordinate'] = crime_df['x_coordinate'].astype(float)
# #crime_df['ward'] = crime_df['ward'].fillna(-1).astype(int)
# #crime_df['year'] = crime_df['year'].fillna(-1).astype(int)

# crime_df['domestic'] = crime_df['domestic'].fillna(-1).astype(int)


# #crime_df['beat'] = crime_df['beat'].fillna(-1).astype(int)
# crime_df['arrest'] = crime_df['arrest'].fillna(-1).astype(int)

# # Convert date and updated_on to strings
# crime_df['date'] = crime_df['date'].astype(str)
# #crime_df['updated_on'] = crime_df['updated_on'].astype(str)
# crime_df['community'] = crime_df['community'].astype(str)

# Now you can insert the DataFrame into the SQL database
crime_df.to_sql('crime_df', conn, if_exists='replace', index=False)

conn.commit()


In [13]:
crime_df.TOT_POP.value_counts()

TOT_POP
97313.0    144287
98505.0    102440
65019.0     85448
54873.0     83794
32106.0     79264
            ...  
13776.0      5931
19477.0      5147
20311.0      4954
2652.0       3962
11494.0      2495
Name: count, Length: 75, dtype: int64

In [11]:
property_df.to_sql('property_df', conn, if_exists='replace', index=False)
conn.commit()


### Feature engieering crime category for each community for each year 

In [147]:
# Convert the year in crime_pivot to integer
crime_pivot.reset_index(inplace=True)
crime_pivot['year'] = crime_pivot['year'].astype(int)

# Ensure the year in property_df is also integer
property_df['year'] = property_df['year'].astype(int)


In [148]:
final_df = pd.merge(property_df, crime_pivot, on=['community', 'year'], how='left')


In [21]:
# Write dataframes to SQL database
property_df.to_sql('property_df', conn, if_exists='replace', index=False)
crime_df.to_sql('crime_df', conn, if_exists='replace', index=False)

# SQL query to calculate the count of crimes for each year
query = """
SELECT community, year, crime_category, COUNT(*) as crime_count
FROM crime_df
GROUP BY community, year, crime_category;

"""
crime_agg = pd.read_sql_query(query, conn)

# Pivot the DataFrame to get community and year as indices and crime categories as columns
#crime_pivot = crime_agg.pivot_table(index=['community', 'year'], columns='crime_category', values='crime_count', fill_value=0)

# Merge with the property data
# Note: Your property_df should also have a 'year' column to merge correctly
#final_df = pd.merge(property_df, crime_pivot, on=['community', 'year'], how='left')


In [23]:
crime_pivot.columns

Index(['Crimes Involving Children', 'Deceptive Practices and Fraud',
       'Drug-Related Crimes', 'Property Crimes', 'Public Order Crimes',
       'Violent Crimes'],
      dtype='object', name='crime_category')

In [24]:
crime_categories = ['Crimes Involving Children', 'Deceptive Practices and Fraud',
       'Drug-Related Crimes', 'Property Crimes', 'Public Order Crimes',
       'Violent Crimes']

In [34]:
import pandas as pd

# First, let's filter out the data for 2023 from the crime_df
crime_df_2023 = crime_df[crime_df['year'] == 2023]

# Now, we need to calculate the sum of each crime category in each community
crime_sum_2023 = crime_df_2023.groupby(['community', 'crime_category'])['crime_category'].count().unstack(fill_value=0)

# Calculate the monthly averages for each crime category
monthly_avg_crime_2023 = crime_sum_2023.div(12).reset_index()

# Add the TOT_POP column to the monthly average DataFrame
monthly_avg_crime_2023 = monthly_avg_crime_2023.merge(
    crime_df_2023[['community', 'TOT_POP']].drop_duplicates(),
    on='community',
    how='left'
)

# Calculate the per capita rates for each crime category
for category in monthly_avg_crime_2023.columns[1:-1]:  # Exclude 'community' and 'TOT_POP'
    monthly_avg_crime_2023[f'{category}_per_capita'] = (monthly_avg_crime_2023[category] / monthly_avg_crime_2023['TOT_POP']) * 100000

# Now you have a DataFrame with monthly average per capita crime rates for each community for the year 2023
monthly_avg_crime_2023 = monthly_avg_crime_2023.set_index('community')

# Optional: Merge this with your property data if needed, ensuring that you have a 'year' column in property_df to merge on.
# Assuming property_df has a 'community' column and a 'year' column for 2023, here's how you could merge them:
final_df = pd.merge(
    property_df[property_df['year'] == 2023],
    monthly_avg_crime_2023,
    on='community',
    how='left'
)


In [35]:
monthly_avg_crime_2023

,Crimes Involving Children,Deceptive Practices and Fraud,Drug-Related Crimes,Property Crimes,Public Order Crimes,Violent Crimes,TOT_POP,Crimes Involving Children_per_capita,Deceptive Practices and Fraud_per_capita,Drug-Related Crimes_per_capita,Property Crimes_per_capita,Public Order Crimes_per_capita,Violent Crimes_per_capita
community,,,,,,,,,,,,,
ALBANY PARK,2.250000,16.583333,2.583333,104.750000,7.083333,63.583333,48647.00000,4.625157,34.089118,5.310365,215.326742,14.560679,130.703503
ARCHER HEIGHTS,0.833333,6.250000,0.500000,40.083333,7.250000,25.250000,13591.00000,6.131509,45.986314,3.678905,294.925563,53.344125,185.784710
ARMOUR SQUARE,0.166667,5.416667,1.000000,51.333333,4.500000,30.416667,13323.66393,1.250907,40.654483,7.505443,385.279407,33.774493,228.290558
ASHBURN,1.916667,16.333333,0.666667,94.333333,11.833333,56.583333,44215.00000,4.334879,36.940706,1.507784,213.351427,26.763165,127.973161
AUBURN GRESHAM,5.916667,23.166667,7.916667,252.916667,40.666667,201.500000,48039.00000,12.316382,48.224706,16.479666,526.481956,84.653441,419.450863
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WEST LAWN,1.333333,9.000000,0.750000,81.083333,8.666667,34.750000,32398.00000,4.115480,27.779493,2.314958,250.272651,26.750622,107.259707
WEST PULLMAN,3.500000,13.666667,4.083333,128.250000,26.916667,94.750000,26951.00000,12.986531,50.709312,15.150953,475.863604,99.872608,351.563949
WEST RIDGE,2.333333,32.333333,3.416667,158.916667,10.333333,90.166667,79897.00000,2.920427,40.468770,4.276339,198.901920,12.933318,112.853632


In [88]:
csv_file_path = 'monthly-avg-cap-crime-2023.csv'

monthly_avg_crime_2023.to_csv(csv_file_path, index=True)


In [18]:
# property_df.to_sql('property_df', conn, if_exists='replace', index=False)
# crime_df.to_sql('crime_df', conn, if_exists='replace', index=False)

# query = """
# SELECT community, crime_category, COUNT(*) as crime_count
# FROM crime_df
# WHERE year BETWEEN 2018 AND 2023
# GROUP BY community, year, crime_category;
# """
# crime_agg = pd.read_sql_query(query, conn)
# crime_pivot = crime_agg.pivot_table(index='community', columns='crime_category', values='crime_count', fill_value=0)

# final_df = pd.merge(property_df, crime_pivot, on='community', how='left')


In [91]:
# Convert 'year' column to datetime format
crime_df['year'] = pd.to_datetime(crime_df['year'])
property_df['year'] = pd.to_datetime(property_df['year'])

# Extract the year from the datetime and update 'year' column
crime_df['year'] = crime_df['year'].dt.year
property_df['year'] = property_df['year'].dt.year


In [12]:
# Write dataframes to SQL database
property_df.to_sql('property_df', conn, if_exists='replace', index=False)
crime_df.to_sql('crime_df', conn, if_exists='replace', index=False)

# SQL query to calculate the count of crimes for each year
query = """
SELECT community, year, crime_category, COUNT(*) as crime_count
FROM crime_df
WHERE year BETWEEN 2014 AND 2023
GROUP BY community, year, crime_category;
"""
crime_agg = pd.read_sql_query(query, conn)

# Pivot the DataFrame to get community and year as indices and crime categories as columns
crime_pivot = crime_agg.pivot_table(index=['community', 'year'], columns='crime_category', values='crime_count', fill_value=0)

# Merge with the property data
# Ensure your property_df has a 'year' column to merge correctly
final_df = pd.merge(property_df, crime_pivot, on=['community', 'year'], how='left')


In [217]:
crime_pivot.head(5)

crime_category    Crimes Involving Children  Deceptive Practices and Fraud  \
community   year                                                             
ALBANY PARK 2014                       16.0                          112.0   
            2015                       20.0                           88.0   
            2016                       25.0                          142.0   
            2017                       25.0                          134.0   
            2018                       14.0                          143.0   

crime_category    Drug-Related Crimes  Property Crimes  Public Order Crimes  \
community   year                                                              
ALBANY PARK 2014                149.0           1092.0                147.0   
            2015                110.0           1257.0                103.0   
            2016                 44.0           1178.0                 67.0   
            2017                 28.0           1298.0                 73.0   
            2018                 33.0           1163.0                 94.0   

crime_category    Violent Crimes  
community   year                  
ALBANY PARK 2014           687.0  
            2015           758.0  
            2016           791.0  
            2017           711.0  
            2018           792.0

In [218]:
csv_file_path = 'crime-categories.csv'

crime_pivot.to_csv(csv_file_path, index=True)


In [188]:
crime_df.head(5)

,date,year,latitude,longitude,description,location_description,updated_on,domestic,primary_type,crime_category,arrest,community,neighborhood,TOT_POP
0,2023-12-31 23:59:00,2023,41.900698,-87.637330,TO VEHICLE,STREET,2024-01-08 15:59:56,False,CRIMINAL DAMAGE,Property Crimes,False,NEAR NORTH SIDE,Central,98505.0
1,2023-12-31 23:40:00,2023,41.892753,-87.624194,AUTOMOBILE,STREET,2024-01-08 15:59:56,False,MOTOR VEHICLE THEFT,Property Crimes,False,NEAR NORTH SIDE,Central,98505.0
2,2023-12-31 23:31:00,2023,41.887919,-87.628019,OTHER VIOLATION,STREET,2024-01-08 15:59:56,False,PUBLIC PEACE VIOLATION,Public Order Crimes,True,NEAR NORTH SIDE,Central,98505.0
3,2023-12-31 22:34:00,2023,41.909959,-87.638555,SIMPLE,APARTMENT,2024-01-08 15:59:56,False,BATTERY,Violent Crimes,False,NEAR NORTH SIDE,Central,98505.0
4,2023-12-31 21:40:00,2023,41.903165,-87.629236,UNLAWFUL POSSESSION - HANDGUN,STREET,2024-01-08 15:59:56,False,WEAPONS VIOLATION,Public Order Crimes,True,NEAR NORTH SIDE,Central,98505.0


### Apporach 2

In [157]:
# Assuming the 'year' in property_df is derived from 'list_date' or a similar column
property_df['list_date'] = pd.to_datetime(property_df['list_date'])
property_df['year'] = property_df['list_date'].dt.year

# Update property_df in the SQL database
property_df.to_sql('property_df', conn, if_exists='replace', index=False)


313125

In [161]:
# Convert the year column in property_df to an integer
property_df['year'] = pd.to_datetime(property_df['last_sold_date']).dt.year

# Also, ensure the year in crime_df is an integer
crime_df['year'] = crime_df['year'].astype(int)

# Save these changes back to your SQL database
property_df.to_sql('property_df', conn, if_exists='replace', index=False)
crime_df.to_sql('crime_df', conn, if_exists='replace', index=False)


2388386

In [167]:
# Ensure the data types are consistent
property_df['community'] = property_df['community'].astype(str)
property_df['year'] = pd.to_datetime(property_df['last_sold_date']).dt.year.astype(int)

crime_df['community'] = crime_df['community'].astype(str)
crime_df['year'] = crime_df['year'].astype(int)

# Save the changes back to the SQL database
property_df.to_sql('property_df', conn, if_exists='replace', index=False)
crime_df.to_sql('crime_df', conn, if_exists='replace', index=False)


2388386

In [192]:
# Execute the SQL query
final_query = """
SELECT 
    p.*, 
    c.crime_year_month, 
    c.Crimes_Involving_Children, 
    c.Deceptive_Practices_and_Fraud, 
    c.Drug_Related_Crimes, 
    c.Property_Crimes, 
    c.Public_Order_Crimes, 
    c.Violent_Crimes
FROM 
    property_df p
LEFT JOIN (
    SELECT 
        community, 
        strftime('%Y-%m', date) as crime_year_month,
        SUM(CASE WHEN crime_category = 'Crimes Involving Children' THEN 1 ELSE 0 END) as Crimes_Involving_Children,
        SUM(CASE WHEN crime_category = 'Deceptive Practices and Fraud' THEN 1 ELSE 0 END) as Deceptive_Practices_and_Fraud,
        SUM(CASE WHEN crime_category = 'Drug-Related Crimes' THEN 1 ELSE 0 END) as Drug_Related_Crimes,
        SUM(CASE WHEN crime_category = 'Property Crimes' THEN 1 ELSE 0 END) as Property_Crimes,
        SUM(CASE WHEN crime_category = 'Public Order Crimes' THEN 1 ELSE 0 END) as Public_Order_Crimes,
        SUM(CASE WHEN crime_category = 'Violent Crimes' THEN 1 ELSE 0 END) as Violent_Crimes
    FROM 
        crime_df
    WHERE 
        strftime('%Y-%m', date) BETWEEN '2014-01' AND '2023-12'
    GROUP BY 
        community, strftime('%Y-%m', date)
) c ON p.community = c.community AND strftime('%Y-%m', p.last_sold_date) = c.crime_year_month
"""
final_df = pd.read_sql_query(final_query, conn)

In [194]:
final_df.head(5)

,property_url,style,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,list_date,sold_price,last_sold_date,lot_sqft,price_per_sqft,latitude,longitude,community,neighborhood,year,crime_year_month,Crimes_Involving_Children,Deceptive_Practices_and_Fraud,Drug_Related_Crimes,Property_Crimes,Public_Order_Crimes,Violent_Crimes
0,https://www.realtor.com/realestateandhomes-detail/8641386055,SINGLE_FAMILY,2.0,1.0,NaN,874.0,1915.0,239.0,42000.0,2013-06-26 00:00:00,59299.0,2014-02-20 00:00:00,NaN,68.0,41.906372,-87.770307,AUSTIN,West Side,2014,2014-02,9.0,41.0,300.0,290.0,67.0,333.0
1,https://www.realtor.com/realestateandhomes-detail/7448076344,MULTI_FAMILY,6.0,2.0,NaN,NaN,1923.0,75.0,245000.0,2013-12-07 00:00:00,244000.0,2014-02-20 00:00:00,3485.0,NaN,41.912893,-87.759807,AUSTIN,West Side,2014,2014-02,9.0,41.0,300.0,290.0,67.0,333.0
2,https://www.realtor.com/realestateandhomes-detail/7039561304,SINGLE_FAMILY,3.0,1.0,NaN,640.0,1903.0,131.0,19900.0,2013-10-11 00:00:00,20500.0,2014-02-19 00:00:00,NaN,32.0,41.890708,-87.748827,AUSTIN,West Side,2014,2014-02,9.0,41.0,300.0,290.0,67.0,333.0
3,https://www.realtor.com/realestateandhomes-detail/7559543149,SINGLE_FAMILY,2.0,2.0,1.0,1200.0,1881.0,544.0,1000.0,2012-08-24 00:00:00,800.0,2014-02-19 00:00:00,3136.0,1.0,41.891081,-87.749859,AUSTIN,West Side,2014,2014-02,9.0,41.0,300.0,290.0,67.0,333.0
4,https://www.realtor.com/realestateandhomes-detail/7530887198,MULTI_FAMILY,6.0,3.0,NaN,NaN,1922.0,134.0,144900.0,2013-10-08 00:00:00,146000.0,2014-02-19 00:00:00,NaN,NaN,41.905976,-87.763646,AUSTIN,West Side,2014,2014-02,9.0,41.0,300.0,290.0,67.0,333.0


In [209]:
# Execute the SQL query
final_query = """
SELECT p.*, c.year AS crime_year, c.Crimes_Involving_Children, c.Deceptive_Practices_and_Fraud, c.Drug_Related_Crimes, c.Property_Crimes, c.Public_Order_Crimes, c.Violent_Crimes
FROM property_df p
LEFT JOIN (
    SELECT community, year,
           SUM(CASE WHEN crime_category = 'Crimes Involving Children' THEN 1 ELSE 0 END) as Crimes_Involving_Children,
           SUM(CASE WHEN crime_category = 'Deceptive Practices and Fraud' THEN 1 ELSE 0 END) as Deceptive_Practices_and_Fraud,
           SUM(CASE WHEN crime_category = 'Drug-Related Crimes' THEN 1 ELSE 0 END) as Drug_Related_Crimes,
           SUM(CASE WHEN crime_category = 'Property Crimes' THEN 1 ELSE 0 END) as Property_Crimes,
           SUM(CASE WHEN crime_category = 'Public Order Crimes' THEN 1 ELSE 0 END) as Public_Order_Crimes,
           SUM(CASE WHEN crime_category = 'Violent Crimes' THEN 1 ELSE 0 END) as Violent_Crimes
    FROM crime_df
    WHERE year BETWEEN 2014 AND 2023
    GROUP BY community, year
) c ON p.community = c.community AND p.year = c.year
"""
final_df = pd.read_sql_query(final_query, conn)


In [212]:
csv_file_path = 'crime-property-crimecat1.csv'

final_df.to_csv(csv_file_path, index=False)

### Creates a new column of monthly average crimes for each community where each row is a crime category with respective stat 

In [26]:
# # Create aggregated crime table
# cursor.execute("""
# CREATE TABLE IF NOT EXISTS monthly_avg_crime_stats_2023 AS
# SELECT 
#     community, 
#     crime_category,
#     CAST(COUNT(*) AS FLOAT) / 12 AS monthly_avg_crimes
# FROM crime_df
# WHERE strftime('%Y', date) = '2023'
# GROUP BY community, crime_category;
# """)

# # Create merged table with property data
# cursor.execute("""
# CREATE TABLE IF NOT EXISTS property_with_monthly_crime_stats AS
# SELECT 
#     property_df.*,
#     monthly_avg_crime_stats_2023.crime_category,
#     monthly_avg_crime_stats_2023.monthly_avg_crimes
# FROM property_df
# LEFT JOIN monthly_avg_crime_stats_2023 
# ON property_df.community = monthly_avg_crime_stats_2023.community;
# """)

### Creates a new column for each crime category that averages the respective monthly count for each neighborhood 

In [125]:
# # Execute SQL commands
# cursor.execute("DROP TABLE IF EXISTS monthly_avg_crime_stats_2023")
# cursor.execute("""CREATE TABLE IF NOT EXISTS monthly_avg_crime_stats_2023 AS
# SELECT 
#     community,
#     MAX(TOT_POP) AS TOT_POP,
#     CAST(SUM(CASE WHEN crime_category = 'Property Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_property_crimes,
#     CAST(SUM(CASE WHEN crime_category = 'Violent Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_violent_crimes,
#     CAST(SUM(CASE WHEN crime_category = 'Deceptive Practices and Fraud' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_deceptive_crimes,
#     CAST(SUM(CASE WHEN crime_category = 'Public Order Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_public_order_crimes,
#     CAST(SUM(CASE WHEN crime_category = 'Drug-Related Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_drug_related_crimes,
#     CAST(SUM(CASE WHEN crime_category = 'Crimes Involving Children' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_crimes_involving_children
# FROM crime_df
# WHERE strftime('%Y', date) = '2023'
# GROUP BY community;
# """)  # The first SQL command

# cursor.execute("""CREATE TABLE IF NOT EXISTS property_with_crime_stats AS
# SELECT 
#     property_df.*,
#     monthly_avg_crime_stats_2023.*
# FROM property_df
# LEFT JOIN monthly_avg_crime_stats_2023 
# ON property_df.community = monthly_avg_crime_stats_2023.community;
# """)  # The second SQL command

# # Commit changes and close the connection
# conn.commit()
# conn.close()


In [131]:
# Execute SQL commands
cursor.execute("DROP TABLE IF EXISTS monthly_avg_crime_stats_2023")
cursor.execute("""CREATE TABLE IF NOT EXISTS monthly_avg_crime_stats_2023 AS
SELECT 
    community,
    MAX(TOT_POP) AS TOT_POP,
    CAST(SUM(CASE WHEN crime_category = 'Property Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_property_crimes,
    CAST(SUM(CASE WHEN crime_category = 'Violent Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_violent_crimes,
    CAST(SUM(CASE WHEN crime_category = 'Deceptive Practices and Fraud' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_deceptive_crimes,
    CAST(SUM(CASE WHEN crime_category = 'Public Order Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_public_order_crimes,
    CAST(SUM(CASE WHEN crime_category = 'Drug-Related Crimes' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_drug_related_crimes,
    CAST(SUM(CASE WHEN crime_category = 'Crimes Involving Children' THEN 1 ELSE 0 END) AS FLOAT) / 12 AS avg_crimes_involving_children
FROM crime_df
WHERE strftime('%Y', date) = '2023'
GROUP BY community;
""")  

cursor.execute("DROP TABLE IF EXISTS property_with_crime_stats")
cursor.execute("""CREATE TABLE IF NOT EXISTS property_with_crime_stats AS
SELECT 
    property_df.*,
    monthly_avg_crime_stats_2023.TOT_POP,
    monthly_avg_crime_stats_2023.avg_property_crimes,
    monthly_avg_crime_stats_2023.avg_violent_crimes,
    monthly_avg_crime_stats_2023.avg_deceptive_crimes,
    monthly_avg_crime_stats_2023.avg_public_order_crimes,
    monthly_avg_crime_stats_2023.avg_drug_related_crimes,
    monthly_avg_crime_stats_2023.avg_crimes_involving_children
FROM property_df
LEFT JOIN monthly_avg_crime_stats_2023 
ON property_df.community = monthly_avg_crime_stats_2023.community;
""")  

conn.commit()
conn.close()

In [83]:
# Query to get the list of tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query and fetch the results
table_names = pd.read_sql_query(query, conn)
print("Tables in the database:", table_names['name'].tolist())


Tables in the database: ['crime_df', 'property_df', 'monthly_avg_crime_stats_2023', 'property_with_crime_stats']


In [77]:
crime_df.head(5)

,date,year,latitude,longitude,description,location_description,updated_on,domestic,primary_type,crime_category,arrest,community,neighborhood,TOT_POP
0,2023-12-31 23:59:00,2023,41.900698,-87.637330,TO VEHICLE,STREET,2024-01-08 15:59:56,False,CRIMINAL DAMAGE,Property Crimes,False,NEAR NORTH SIDE,Central,98505.0
1,2023-12-31 23:40:00,2023,41.892753,-87.624194,AUTOMOBILE,STREET,2024-01-08 15:59:56,False,MOTOR VEHICLE THEFT,Property Crimes,False,NEAR NORTH SIDE,Central,98505.0
2,2023-12-31 23:31:00,2023,41.887919,-87.628019,OTHER VIOLATION,STREET,2024-01-08 15:59:56,False,PUBLIC PEACE VIOLATION,Public Order Crimes,True,NEAR NORTH SIDE,Central,98505.0
3,2023-12-31 22:34:00,2023,41.909959,-87.638555,SIMPLE,APARTMENT,2024-01-08 15:59:56,False,BATTERY,Violent Crimes,False,NEAR NORTH SIDE,Central,98505.0
4,2023-12-31 21:40:00,2023,41.903165,-87.629236,UNLAWFUL POSSESSION - HANDGUN,STREET,2024-01-08 15:59:56,False,WEAPONS VIOLATION,Public Order Crimes,True,NEAR NORTH SIDE,Central,98505.0


In [84]:


# Define and execute the SQL query to fetch the first 5 rows of the table
query = "SELECT * FROM 'property_with_crime_stats' LIMIT 5"
cursor.execute(query)

# Fetch the results and print them
results = cursor.fetchall()

for row in results:
    print(row)

# Close the cursor and the database connection when done
cursor.close()
conn.close()


('https://www.realtor.com/realestateandhomes-detail/8641386055', 'SINGLE_FAMILY', 2.0, 1.0, None, 874.0, 1915.0, 239.0, 42000.0, '2013-06-26', 59299.0, '2014-02-20', None, 68.0, 41.906372, -87.770307, 'AUSTIN', 'West Side', 'AUSTIN', 406.25, 384.1666666666667, 47.416666666666664, 77.75, 38.25, 7.333333333333333)
('https://www.realtor.com/realestateandhomes-detail/7448076344', 'MULTI_FAMILY', 6.0, 2.0, None, None, 1923.0, 75.0, 245000.0, '2013-12-07', 244000.0, '2014-02-20', 3485.0, None, 41.912893, -87.759807, 'AUSTIN', 'West Side', 'AUSTIN', 406.25, 384.1666666666667, 47.416666666666664, 77.75, 38.25, 7.333333333333333)
('https://www.realtor.com/realestateandhomes-detail/7039561304', 'SINGLE_FAMILY', 3.0, 1.0, None, 640.0, 1903.0, 131.0, 19900.0, '2013-10-11', 20500.0, '2014-02-19', None, 32.0, 41.890708, -87.748827, 'AUSTIN', 'West Side', 'AUSTIN', 406.25, 384.1666666666667, 47.416666666666664, 77.75, 38.25, 7.333333333333333)
('https://www.realtor.com/realestateandhomes-detail/75595

In [133]:
import pandas as pd

# Set option to display all columns
pd.set_option('display.max_columns', None)

# Your existing code
table_name = 'property_with_crime_stats'
query = f"SELECT * FROM {table_name} LIMIT 10;"
table_data = pd.read_sql_query(query, conn)
print(f"First 10 rows of the table {table_name}:")
display(table_data)


First 10 rows of the table property_with_crime_stats:


,property_url,style,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,list_date,sold_price,last_sold_date,lot_sqft,price_per_sqft,latitude,longitude,community,neighborhood,TOT_POP,avg_property_crimes,avg_violent_crimes,avg_deceptive_crimes,avg_public_order_crimes,avg_drug_related_crimes,avg_crimes_involving_children
0,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,2.0,1.0,NaN,874.0,1915.0,239.0,42000.0,2013-06-26,59299.0,2014-02-20,NaN,68.0,41.906372,-87.770307,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
1,https://www.realtor.com/realestateandhomes-det...,MULTI_FAMILY,6.0,2.0,NaN,NaN,1923.0,75.0,245000.0,2013-12-07,244000.0,2014-02-20,3485.0,NaN,41.912893,-87.759807,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
2,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,3.0,1.0,NaN,640.0,1903.0,131.0,19900.0,2013-10-11,20500.0,2014-02-19,NaN,32.0,41.890708,-87.748827,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
3,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,2.0,2.0,1.0,1200.0,1881.0,544.0,1000.0,2012-08-24,800.0,2014-02-19,3136.0,1.0,41.891081,-87.749859,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
4,https://www.realtor.com/realestateandhomes-det...,MULTI_FAMILY,6.0,3.0,NaN,NaN,1922.0,134.0,144900.0,2013-10-08,146000.0,2014-02-19,NaN,NaN,41.905976,-87.763646,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
5,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,3.0,1.0,NaN,1367.0,1917.0,NaN,NaN,None,69000.0,2014-02-14,5200.0,50.0,41.892542,-87.773728,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
6,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,2.0,1.0,NaN,1000.0,1890.0,NaN,NaN,None,55000.0,2014-02-03,3000.0,55.0,41.896851,-87.771882,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
7,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,5.0,3.0,NaN,1700.0,1915.0,71.0,235000.0,2013-11-21,231000.0,2014-01-31,NaN,136.0,41.908181,-87.767945,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
8,https://www.realtor.com/realestateandhomes-det...,MULTI_FAMILY,5.0,2.0,NaN,NaN,1920.0,88.0,123900.0,2013-11-04,157500.0,2014-01-31,4792.0,NaN,41.888672,-87.771230,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
9,https://www.realtor.com/realestateandhomes-det...,CONDO_TOWNHOME_ROWHOME_COOP,2.0,1.0,NaN,1450.0,1930.0,149.0,50500.0,2013-09-04,50000.0,2014-01-31,NaN,34.0,41.888679,-87.773416,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333


In [134]:
# Specify the table name you are interested in
table_name = 'property_with_crime_stats'
# Query to fetch the first 10 rows of the specified table
query = f"SELECT * FROM {table_name} LIMIT 10;"

# Execute the query and fetch the results
table_data = pd.read_sql_query(query, conn)
print(f"First 10 rows of the table {table_name}:")
display(table_data)

First 10 rows of the table property_with_crime_stats:


,property_url,style,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,list_date,sold_price,last_sold_date,lot_sqft,price_per_sqft,latitude,longitude,community,neighborhood,TOT_POP,avg_property_crimes,avg_violent_crimes,avg_deceptive_crimes,avg_public_order_crimes,avg_drug_related_crimes,avg_crimes_involving_children
0,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,2.0,1.0,NaN,874.0,1915.0,239.0,42000.0,2013-06-26,59299.0,2014-02-20,NaN,68.0,41.906372,-87.770307,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
1,https://www.realtor.com/realestateandhomes-det...,MULTI_FAMILY,6.0,2.0,NaN,NaN,1923.0,75.0,245000.0,2013-12-07,244000.0,2014-02-20,3485.0,NaN,41.912893,-87.759807,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
2,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,3.0,1.0,NaN,640.0,1903.0,131.0,19900.0,2013-10-11,20500.0,2014-02-19,NaN,32.0,41.890708,-87.748827,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
3,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,2.0,2.0,1.0,1200.0,1881.0,544.0,1000.0,2012-08-24,800.0,2014-02-19,3136.0,1.0,41.891081,-87.749859,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
4,https://www.realtor.com/realestateandhomes-det...,MULTI_FAMILY,6.0,3.0,NaN,NaN,1922.0,134.0,144900.0,2013-10-08,146000.0,2014-02-19,NaN,NaN,41.905976,-87.763646,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
5,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,3.0,1.0,NaN,1367.0,1917.0,NaN,NaN,None,69000.0,2014-02-14,5200.0,50.0,41.892542,-87.773728,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
6,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,2.0,1.0,NaN,1000.0,1890.0,NaN,NaN,None,55000.0,2014-02-03,3000.0,55.0,41.896851,-87.771882,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
7,https://www.realtor.com/realestateandhomes-det...,SINGLE_FAMILY,5.0,3.0,NaN,1700.0,1915.0,71.0,235000.0,2013-11-21,231000.0,2014-01-31,NaN,136.0,41.908181,-87.767945,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
8,https://www.realtor.com/realestateandhomes-det...,MULTI_FAMILY,5.0,2.0,NaN,NaN,1920.0,88.0,123900.0,2013-11-04,157500.0,2014-01-31,4792.0,NaN,41.888672,-87.771230,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333
9,https://www.realtor.com/realestateandhomes-det...,CONDO_TOWNHOME_ROWHOME_COOP,2.0,1.0,NaN,1450.0,1930.0,149.0,50500.0,2013-09-04,50000.0,2014-01-31,NaN,34.0,41.888679,-87.773416,AUSTIN,West Side,97313.0,406.25,384.166667,47.416667,77.75,38.25,7.333333


In [135]:
query = "SELECT * FROM monthly_avg_crime_stats_2023 LIMIT 10;"
monthly_avg_data = pd.read_sql_query(query, conn)
print("First 10 rows of monthly_avg_crime_stats_2023:")
display(monthly_avg_data)


First 10 rows of monthly_avg_crime_stats_2023:


,community,TOT_POP,avg_property_crimes,avg_violent_crimes,avg_deceptive_crimes,avg_public_order_crimes,avg_drug_related_crimes,avg_crimes_involving_children
0,ALBANY PARK,48647.00000,104.750000,63.583333,16.583333,7.083333,2.583333,2.250000
1,ARCHER HEIGHTS,13591.00000,40.083333,25.250000,6.250000,7.250000,0.500000,0.833333
2,ARMOUR SQUARE,13323.66393,51.333333,30.416667,5.416667,4.500000,1.000000,0.166667
3,ASHBURN,44215.00000,94.333333,56.583333,16.333333,11.833333,0.666667,1.916667
4,AUBURN GRESHAM,48039.00000,252.916667,201.500000,23.166667,40.666667,7.916667,5.916667
5,AUSTIN,97313.00000,406.250000,384.166667,47.416667,77.750000,38.250000,7.333333
6,AVALON PARK,9740.26656,45.166667,29.250000,4.833333,5.333333,0.416667,0.583333
7,AVONDALE,36820.00000,98.000000,48.666667,11.500000,6.583333,1.666667,0.583333
8,BELMONT CRAGIN,78610.00000,176.416667,131.833333,33.250000,20.666667,5.666667,3.666667
9,BEVERLY,19919.00000,42.166667,19.000000,8.666667,3.333333,0.750000,1.166667


In [116]:
query = "SELECT * FROM crime_df LIMIT 10;"
crime_data = pd.read_sql_query(query, conn)
print("First 10 rows of crime_df:")
display(crime_data)


First 10 rows of crime_df:


,date,year,latitude,longitude,description,location_description,updated_on,domestic,primary_type,crime_category,arrest,community,neighborhood,TOT_POP
0,2023-12-31 23:59:00,2023,41.900698,-87.637330,TO VEHICLE,STREET,2024-01-08 15:59:56,0,CRIMINAL DAMAGE,Property Crimes,0,NEAR NORTH SIDE,Central,98505.0
1,2023-12-31 23:40:00,2023,41.892753,-87.624194,AUTOMOBILE,STREET,2024-01-08 15:59:56,0,MOTOR VEHICLE THEFT,Property Crimes,0,NEAR NORTH SIDE,Central,98505.0
2,2023-12-31 23:31:00,2023,41.887919,-87.628019,OTHER VIOLATION,STREET,2024-01-08 15:59:56,0,PUBLIC PEACE VIOLATION,Public Order Crimes,1,NEAR NORTH SIDE,Central,98505.0
3,2023-12-31 22:34:00,2023,41.909959,-87.638555,SIMPLE,APARTMENT,2024-01-08 15:59:56,0,BATTERY,Violent Crimes,0,NEAR NORTH SIDE,Central,98505.0
4,2023-12-31 21:40:00,2023,41.903165,-87.629236,UNLAWFUL POSSESSION - HANDGUN,STREET,2024-01-08 15:59:56,0,WEAPONS VIOLATION,Public Order Crimes,1,NEAR NORTH SIDE,Central,98505.0
5,2023-12-31 20:45:00,2023,41.896683,-87.627860,SIMPLE,RESTAURANT,2024-01-08 15:59:56,0,ASSAULT,Violent Crimes,0,NEAR NORTH SIDE,Central,98505.0
6,2023-12-31 20:02:00,2023,41.891926,-87.628074,MANUFACTURE / DELIVER - CANNABIS OVER 10 GRAMS,SIDEWALK,2024-01-08 15:59:56,0,NARCOTICS,Drug-Related Crimes,1,NEAR NORTH SIDE,Central,98505.0
7,2023-12-31 20:00:00,2023,41.890052,-87.628914,SIMPLE,STREET,2024-01-08 15:59:56,0,ASSAULT,Violent Crimes,1,NEAR NORTH SIDE,Central,98505.0
8,2023-12-31 20:00:00,2023,41.892017,-87.624165,DOMESTIC BATTERY SIMPLE,HOTEL / MOTEL,2024-01-08 15:59:56,1,BATTERY,Violent Crimes,1,NEAR NORTH SIDE,Central,98505.0
9,2023-12-31 19:37:00,2023,41.889414,-87.631066,UNLAWFUL POSSESSION - HANDGUN,STREET,2024-01-08 15:59:56,0,WEAPONS VIOLATION,Public Order Crimes,1,NEAR NORTH SIDE,Central,98505.0
